## DarkSky APi to InfluxDB

In [99]:
import pandas as pd

In [115]:
coords = (45.1973288, 5.7139923)
day = '2017-10-04T00:00:00'

In [116]:
from darksky import forecast

# """ You get 1,000 API calls for free each day. """

# Load the API key for darksky
with open('darksky_key.txt') as f:
    KEY = f.read()

EXCLUDE = ['currently', 'minutely', 'daily', 'flags']  # from the query

In [236]:
def query2dataframe( coords,  day ):
    # query
    data = forecast(KEY, *coords, units='si', lang='fr', time=day, exclude=EXCLUDE)
    
    print('get data for %s' % day)
    
    # build the DataFrame from records (dict)
    records = data['hourly']['data']  
    weatherdataDF = pd.DataFrame.from_records(records, index='time')

    weatherdataDF.index = pd.to_datetime(weatherdataDF.index, unit='s')
    
    # debug: force uvIndex to float  ( plutôt que Int à cause des NaN )  precipIntensity
    
    cols_int = list( weatherdataDF.select_dtypes(include=['int']).columns )
    for c in cols_int:
        weatherdataDF[c] = weatherdataDF[c].dropna().astype(float)
        
    return weatherdataDF

In [211]:
from influxdb import DataFrameClient

In [238]:
# see https://influxdb-python.readthedocs.io/en/latest/examples.html#tutorials-pandas
"""Instantiate the connection to the InfluxDB client."""

host, port = 'localhost', 8086
user, password = 'root', 'root'
dbname = 'weatherdata'
protocol = 'line'

clientDB = DataFrameClient(host, port, user, password, dbname)


print("Create database: " + dbname)
clientDB.create_database(dbname)

measurement_name = 'darksky'

Create database: weatherdata


In [237]:
# delete
client.drop_database( dbname  )

In [197]:
def dataFrame2influx( client, DF ):
    # Ajoute colone par colones pour retirer les NaN...
    for c in DF.columns:
        client.write_points(DF[[c]].dropna(), measurement_name)
        
def add1day( client, coords, day ):
    weatherdataDF = query2dataframe( coords,  day )
    dataFrame2influx( client, weatherdataDF )

In [251]:
firstday = '2017-09-01'
lastday = '2017-09-22'
daterange = pd.date_range(start=firstday, end=lastday,  freq='D', normalize=True)
daterange_iso = daterange.strftime('%Y-%m-%dT%H:%M:%SZ')

In [252]:
for d in daterange_iso:
    add1day( client, coords, d )

get data for 2017-09-01T00:00:00Z
get data for 2017-09-02T00:00:00Z
get data for 2017-09-03T00:00:00Z
get data for 2017-09-04T00:00:00Z
get data for 2017-09-05T00:00:00Z
get data for 2017-09-06T00:00:00Z
get data for 2017-09-07T00:00:00Z
get data for 2017-09-08T00:00:00Z
get data for 2017-09-09T00:00:00Z
get data for 2017-09-10T00:00:00Z
get data for 2017-09-11T00:00:00Z
get data for 2017-09-12T00:00:00Z
get data for 2017-09-13T00:00:00Z
get data for 2017-09-14T00:00:00Z
get data for 2017-09-15T00:00:00Z
get data for 2017-09-16T00:00:00Z
get data for 2017-09-17T00:00:00Z
get data for 2017-09-18T00:00:00Z
get data for 2017-09-19T00:00:00Z
get data for 2017-09-20T00:00:00Z
get data for 2017-09-21T00:00:00Z
get data for 2017-09-22T00:00:00Z


In [255]:
weatherdataDF = query2dataframe( coords,  '2017-09-02T00:00:00Z' )

get data for 2017-09-02T00:00:00Z


In [256]:
weatherdataDF

,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,precipType,pressure,summary,temperature,uvIndex,visibility,windBearing,windSpeed
time,,,,,,,,,,,,,,,
2017-09-01 22:00:00,13.90,0.80,10.55,0.80,rain,0.2134,0.42,rain,1018.79,Bruine,13.90,0.0,12.94,324.0,0.72
2017-09-01 23:00:00,13.33,0.87,10.84,0.85,partly-cloudy-night,0.1194,0.47,rain,1018.74,Nuageux,13.33,0.0,12.94,318.0,1.39
2017-09-02 00:00:00,13.03,0.75,10.88,0.87,partly-cloudy-night,0.0000,0.00,NaN,1018.58,Nuageux,13.03,0.0,12.25,312.0,1.36
2017-09-02 01:00:00,12.91,0.85,10.61,0.86,partly-cloudy-night,0.0000,0.00,NaN,1018.37,Nuageux,12.91,0.0,12.94,271.0,1.01
2017-09-02 02:00:00,12.29,0.92,10.36,0.88,partly-cloudy-night,0.0000,0.00,NaN,1018.52,Nuageux,12.29,0.0,12.25,304.0,1.22
2017-09-02 03:00:00,12.50,0.96,10.59,0.88,cloudy,0.0000,0.00,NaN,1018.38,Couvert,12.50,0.0,11.62,349.0,1.35
2017-09-02 04:00:00,11.09,0.96,9.43,0.90,cloudy,0.0660,0.13,rain,1018.30,Couvert,11.09,0.0,11.62,353.0,1.48
2017-09-02 05:00:00,12.55,0.93,10.37,0.87,partly-cloudy-night,0.1245,0.18,rain,1018.34,Nuageux,12.55,0.0,11.54,344.0,1.48
2017-09-02 06:00:00,13.01,0.96,10.65,0.86,cloudy,0.0000,0.00,NaN,1018.35,Couvert,13.01,0.0,11.44,355.0,1.89


## EmonCMS

In [257]:
import emoncmsfeed as getfeeds

In [269]:
startday = pd.to_datetime('2/08/2017', format='%d/%m/%Y').tz_localize('Europe/Paris')
lastday = pd.to_datetime('5/08/2017', format='%d/%m/%Y').tz_localize('Europe/Paris')

In [270]:
dataframefreq = '15min'
feeds =  { 'T_int': 3 }

Tmesure = getfeeds.builddataframe( feeds, dataframefreq , startdate=startday, enddate=lastday )
Tmesure.plot( figsize=(14, 3) );

In [271]:
# see https://influxdb-python.readthedocs.io/en/latest/examples.html#tutorials-pandas
"""Instantiate the connection to the InfluxDB client."""

host, port = 'localhost', 8086
user, password = 'root', 'root'
dbname = 'weatherdata'
protocol = 'line'

clientDB = DataFrameClient(host, port, user, password, dbname)


print("Create database: " + dbname)
clientDB.create_database(dbname)

measurement_name = 'openenergymonitor'

Create database: weatherdata


In [272]:
DF = Tmesure
# Ajoute colone par colones pour retirer les NaN...
for c in DF.columns:
    client.write_points(DF[[c]].dropna(), measurement_name)